In [1]:
from z3 import *

# Distribuição de gabinetes

Considere que temos 3 gabinetes e queremos distribuir 4 pessoas (Ana=1, Nuno=2, Pedro=3 e Maria=4) por esses gabinetes.

Considere ainda que foram estipuladas as seguintes regras de ocupação dos gabinetes:

1. Cada pessoa ocupa um único gabinete.
2. O Nuno e o Pedro não podem partilhar gabinete.
3. Se a Ana ficar sozinha num gabinete, então o Pedro também terá que ficar sozinho num gabinete.
4. Cada gabinete só pode acomodar, no máximo, 2 pessoas.

Pretende-se que escreva um programa Python que, usando o Z3 como SAT solver, faça a distribuição das pessoas pelos gabinetes.

Começe por instalar o Z3.


Para resolver o problema em Lógica Proposicional temos que

1. Declarar um conjunto de variáveis Boolenas
$x_{p,g}$ com a seguinte semântica:

  > $x_{p,g}$ é verdade  *sse*   a pessoa  $p$ ocupa o gabinete $g$

2. De seguida, teremos que modelar cada uma das restrições, acrescentando as fórmulas lógicas correspondentes.

3. Finalmente testamos se o conjunto de restrições é satisfazível e extraimos a solução calculada.  


In [2]:
pessoas = ["Ana","Nuno","Pedro","Maria"]
gabs    = [1,2,3]
x = {}
for p in pessoas:
    x[p] = {}
    for g in gabs:
        x[p][g] = Bool("%s,%d" % (p,g))

s = Solver()

# Cada pessoa ocupa pelo menos um gabinete.
for p in pessoas:
    s.add( Or([x[p][g] for g in gabs ]) )

# Cada pessoa ocupa no máximo um gabinete.
for p in pessoas:
    for g in gabs:
        s.add( [Implies( x[p][g], Not(x[p][gi]) ) for gi in gabs if  gi != g ] )

# O Nuno e o Pedro não podem partilhar gabinete.
for g in gabs:
    s.add( Implies(x["Pedro"][g], Not(x["Nuno"][g]) ) )
    
# Se a Ana ficar sozinha num gabinete, então o Pedro também terá que ficar sozinho num gabinete.

def sozinho(p):
    res = []
    for g in gabs:
        res.append ( Implies ( x[p][g], And( [Not(x[pi][g]) for pi in pessoas if  pi != p] ) ) )   
    return And(res)

s.add( Implies( sozinho("Ana"), sozinho("Pedro") ) )

# Cada gabinete só pode acomodar, no máximo, 2 pessoas.
for g in gabs:
    for p0 in pessoas:
        for p1 in [pi for pi in pessoas if pi != p0]:
                s.add(Implies( 
                    
                    And(x[p0][g], x[p1][g]), 
                    
                    And([ Not (x[p2][g]) for p2 in [pi for pi in pessoas if pi != p0 and pi != p1]])  
                            )
                     )
                
s.push()
if s.check() == sat:
    m = s.model()
    for p in pessoas:
        for g in gabs:
            if is_true(m[x[p][g]]):
                print("%s fica no gabinete %s" % (p,g))
else:
  print("Não tem solução.")

Ana fica no gabinete 2
Nuno fica no gabinete 2
Pedro fica no gabinete 1
Maria fica no gabinete 3


Use agora o SAT solver para testar a veracidade se cada uma das seguintes afirmações:

1. Se a Maria ocupar o gabinete um, então ela ficará sozinha nesse gabinete.
2. Se a Ana e o Nuno ficarem no mesmo gabinete, então a Maria e o Pedro terão que partilhar também um outro gabinete.

A que conclusões chegou?

In [3]:
#Exercicio 1
#Se a Maria ocupar o gabinete um, então alguém tem de ficar nesse gabinete.
s.add( Implies (x["Maria"][1], Or( x["Pedro"][1], x["Nuno"][1], x["Ana"][1] ) ) )
s.add(x["Maria"][1])
s.push()

if s.check() == sat:
    m = s.model()
    for p in pessoas:
        for g in gabs:
            if is_true(m[x[p][g]]):
                print("%s fica no gabinete %s" % (p,g))
    
    print("A afirmação é falsa")

else:
    print("Não tem solução.")
    print("A afirmação é verdade")

Ana fica no gabinete 1
Nuno fica no gabinete 2
Pedro fica no gabinete 3
Maria fica no gabinete 1
A afirmação é falsa


In [4]:
#Exercicio 2
#Se a Ana e o Nuno ficarem no mesmo gabinete então a Maria e o Pedro terão que ficar em gabinetes separados
s.pop()

def juntos(p1,p2):
    res = []
    for g in gabs:
        s.add( Implies(x[p1][g], x[p2][g]) ) 
    return And(res)

s.add( Implies( juntos("Ana","Nuno"), And( sozinho("Maria"), sozinho("Pedro") ) ) )
s.add(juntos("Ana","Nuno"))

s.push()

if s.check() == sat:
    m = s.model()
    for p in pessoas:
        for g in gabs:
            if is_true(m[x[p][g]]):
                print("%s fica no gabinete %s" % (p,g))
    print("A afirmação é falsa")
    
else:
    print("Não tem solução.")
    print("A afirmação é verdade")

Não tem solução.
A afirmação é verdade
